In [1]:
import boto3
import sagemaker
from sagemaker import image_uris
import boto3
import os
import time
import json
import urllib.request
from pathlib import Path
from sagemaker.utils import name_from_base

role = sagemaker.get_execution_role()  # execution role for the endpoint
sess = sagemaker.session.Session()  # sagemaker session for interacting with different AWS APIs
default_bucket = sess.default_bucket()  # bucket to house artifacts
region = sess._region_name

default_bucket


https://proteindc-neptune-db-cluster.cluster-cnfpwwtkftli.us-east-1.neptune.amazonaws.com:8182/loader


In [ ]:
neptune_host = 'proteindc-neptune-db-cluster.cluster-cnfpwwtkftli.us-east-1.neptune.amazonaws.com'
neptune_port = '8182'
iamRoleArn = 'arn:aws:iam::629244530291:role/zk-NeptuneLoadFromS3'

In [ ]:

url = f'https://{neptune_host}:{neptune_port}/loader'
graph_notebook_host = f'https://{neptune_host}'

%graph_notebook_host {neptune_host}

In [2]:


source_bucket_name = 'aws-open-data-uniprot-rdf'
bucket_name = source_bucket_name
if(region != 'eu-west-3'):
    bucket_name = default_bucket

print(bucket_name)
# define a list of files to be loaded
files_to_load = [
    "2023-03/supporting/taxonomy.rdf.gz",
    "2023-03/supporting/go.rdf.gz",
    "2023-03/uniprot/uniprotkb_reviewed_eukaryota_opisthokonta_metazoa_33208_0.rdf.gz"
]



In [ ]:
s3 = boto3.resource('s3')

if(region != 'eu-west-3'):
    for file in files_to_load:
        copy_source = {
            'Bucket': source_bucket_name,
            'Key': file
         }
        s3.meta.client.copy(copy_source, default_bucket, file)


In [4]:
loadids = []
print(url)
def loadfile(filelocation):
    print(f"s3://{bucket_name}/{filelocation}")
    print(iamRoleArn)
    print(region)
    data = {
      "source" : f"s3://{bucket_name}/{filelocation}",
      "format" : "rdfxml",
      "iamRoleArn" : iamRoleArn,
      "region" : region,
      "failOnError" : "FALSE",
      "parallelism" : "OVERSUBSCRIBE",
      "queueRequest" : "TRUE"
    }

    data = json.dumps(data)
    print(data)

    req = urllib.request.Request(url = url, data = bytes(data.encode("utf-8")), method = "POST")

    req.add_header("Content-type", "application/json; charset=UTF-8")

    with urllib.request.urlopen(req) as resp:
        response_data = json.loads(resp.read().decode("utf-8"))
        loadId=response_data['payload']['loadId']
        
    print("load id: {}".format(loadId))
    loadids.append(loadId)
    return loadids
    
for file in files_to_load:
    loadids = loadfile(file)

https://proteindc-neptune-db-cluster.cluster-cnfpwwtkftli.us-east-1.neptune.amazonaws.com:8182/loader
s3://aws-open-data-uniprot-rdf/2021-01/supporting/taxonomy.rdf.gz
{"source": "s3://aws-open-data-uniprot-rdf/2021-01/supporting/taxonomy.rdf.gz", "format": "rdfxml", "iamRoleArn": "arn:aws:iam::629244530291:role/zk-NeptuneLoadFromS3", "region": "us-east-1", "failOnError": "FALSE", "parallelism": "OVERSUBSCRIBE", "queueRequest": "TRUE"}


HTTPError: HTTP Error 400: Bad Request

# 监控数据加载进度

In [6]:
import time
for load_id in loadids:
    req = urllib.request.Request(url = "/".join([url, load_id]), method = "GET")
    req.add_header("Content-type", "application/json; charset=UTF-8")
    
    status = None
    while status != 'LOAD_COMPLETED':
        with urllib.request.urlopen(req) as resp:
            response_data = json.loads(resp.read().decode("utf-8"))['payload']

        status = response_data['overallStatus']['status']
        totalTimeSpent = response_data['overallStatus']['totalTimeSpent']
        totalRecords = response_data['overallStatus']['totalRecords']
        end = '\n' if status == 'LOAD_COMPLETED' else '\r'
        print(f"{load_id}  status: {status} \tload time: {totalTimeSpent}s\trecords: {totalRecords}", end=end)
        time.sleep(1)

6c085caf-c274-43b0-9955-1b2b66a54f33  status: LOAD_COMPLETED 	load time: 488s	records: 15950018
ebfd5eb5-52c9-47c0-889a-7e1c1615b9af  status: LOAD_COMPLETED 	load time: 17s	records: 38139292
aa57dad4-97df-4caf-810f-72057d72adfd  status: LOAD_COMPLETED 	load time: 10394s	records: 34441224300


#  测试

In [ ]:
%%sparql
SELECT ?s ?p ?o WHERE {?s ?p ?o} LIMIT 10